In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,GridSearchCV #divide dataset and find the best estimator
from sklearn.preprocessing import StandardScaler #standardise dataset
from sklearn.neighbors import KNeighborsClassifier  #classify dataset

Specific steps
* Access to datset
* Basic dataset processing
    * Narrowing the data
    * Selecte valid time features
    * Removal of less signed-in areas(去除签到较少的地方)
    * Determining feature values and target values
* Dividing the dataset 
* Standardising the dataset
* Machine Learning(Model traing)
* Model evaluation(模型评估)

In [2]:
#1 Access to dataset
feacbook_train=pd.read_csv("train-facebook.csv")
feacbook_train.head()

,row_id,x,y,accuracy,time,place_id
0,0,0.7941,9.0809,54,470702,8523065625
1,1,5.9567,4.7968,13,186555,1757726713
2,2,8.3078,7.0407,74,322648,1137537235
3,3,7.3665,2.5165,65,704587,6567393236
4,4,4.0961,1.1307,31,472130,7440663949


In [3]:
#2.1 Narrowing the data（缩小数据范围）
facebook_data=feacbook_train.query("x>2 & x<2.5 & y>2 & y<2.5") #select data range
#2.2 Select valid time feature（选择有效的时间）
time=pd.to_datetime(facebook_data["time"],unit="s")
time=pd.DatetimeIndex(time)  #converting time data into a list （将时间存放之列表）
facebook_data["day"]=time.day
facebook_data["hour"]=time.hour
facebook_data["year"]=time.year
facebook_data.head()
#2.3 Removal of less signed-in areas
place_count=facebook_data.groupby(["place_id"]).count()
place_count=place_count[place_count["row_id"]>3]  #number of signed-ins >3
facebook_data=facebook_data[facebook_data["place_id"].isin(place_count.index)]
facebook_data.head()
#2.4 Determining feature values and target values
x=facebook_data[["x","y","accuracy","day","hour","year"]]  
label=facebook_data["place_id"]

D:\anaconda33\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\anaconda33\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
D:\anaconda33\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

In [5]:
#3 Dividing the data（切割数据）
x_train,x_test,y_train,y_test=train_test_split(x,label,random_state=22)

#4 Standardising the data（数据标准化）
transfer=StandardScaler()
x_train=transfer.fit_transform(x_train)
x_test=transfer.transform(x_test)

#5 Model training（模型训练）
estimator=KNeighborsClassifier()
'''several fold cross-validation'''
param_dict={"n_neighbors":[1,3,5,6,7,9]}
estimator=GridSearchCV(estimator,param_grid=param_dict,cv=5)
estimator.fit(x_train,y_train)  #traing the model
estimator=estimator.best_estimator_ #the best estimator

#6 Model evaluation
y_predict=estimator.predict(x_test)
print("Compare actual results with predicted results\n",y_predict==y_test)
accuracy=estimator.score(x_test,y_test)
print("accuracy={:.2}".format(accuracy))


D:\anaconda33\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


Compare actual results with predicted results
 24703810     True
19445902    False
18490063     True
7762709     False
6505956     False
            ...  
27632888    False
23367671    False
6692268     False
25834435    False
13319005     True
Name: place_id, Length: 17316, dtype: bool
accuracy=0.38
